# Opole Real Estate Market – Data Cleaning

This notebook contains the data cleaning and preparation steps for apartment listings
collected from Otodom.pl for the city of Opole.

The goal of this step is to transform the raw scraped data into a clean, structured
dataset suitable for further analysis and visualization in Power BI.

### Key steps in this notebook:
- load raw scraped data
- remove duplicate listings using a technical identifier
- convert textual fields (price, area) to numeric values
- perform basic validation and prepare the dataset for analysis


In [1]:
import pandas as pd

df = pd.read_csv("../data/raw/opole_listings_raw.csv")

df.head()

,link,price,location,area,price_per_m2,rooms,floor,offer_type
0,https://www.otodom.pl/pl/oferta/zaodrze-po-gen...,528 000 zł,"ul. Alojzego Dambonia, Zaodrze, Opole, opolskie",47 m²,11 234 zł/m²,2 pokoje,2 piętro,Oferta prywatna
1,https://www.otodom.pl/pl/oferta/przestronne-mi...,525 000 zł,"ul. Powstańców Śląskich, Stare Miasto, Opole, ...",46.88 m²,11 199 zł/m²,1 pokój,1 piętro,Oferta prywatna
2,https://www.otodom.pl/pl/oferta/apartament-w-s...,695 943 zł,"ul. o. Józefa Czaplaka, Śródmieście, Opole, op...",47.7 m²,14 590 zł/m²,4 pokoje,1 piętro,Developer
3,https://www.otodom.pl/pl/oferta/65m2-z-ogrodem...,658 000 zł,"Chmielowice, Opole, opolskie",65.5 m²,10 046 zł/m²,3 pokoje,parter,Developer
4,https://www.otodom.pl/pl/oferta/2-pokojowe-mie...,564 456 zł,"ul. Pomorska/Zielonogórska, Kolonia Gosławicka...",48.66 m²,11 600 zł/m²,2 pokoje,1 piętro,Developer


## 1. Initial data overview

Before cleaning, we inspect the structure, size, and basic information
about the raw dataset.

In [2]:
df.shape

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566 entries, 0 to 565
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   link          566 non-null    object
 1   price         551 non-null    object
 2   location      551 non-null    object
 3   area          551 non-null    object
 4   price_per_m2  551 non-null    object
 5   rooms         551 non-null    object
 6   floor         515 non-null    object
 7   offer_type    553 non-null    object
dtypes: object(8)
memory usage: 35.5+ KB


## 2. Remove duplicate listings

Some listings appear multiple times across pages due to promotion.
Duplicates are removed using the listing URL as a technical identifier.

In [3]:
initial_rows = df.shape[0]

df = df.drop_duplicates(subset="link")

after_rows = df.shape[0]

initial_rows, after_rows

(566, 517)

## 3. Price cleaning

Convert the price field from text format (e.g. "528 000 zł")
to a numeric value for analysis.

In [4]:
NON_NUMERIC_PRICE_PATTERN = r"^(?:od|za|zapytaj|zadzwoń|do uzgodnienia)"

df = df[df["price"].notna()]
df["price"] = df["price"].astype(str)

df = df[~df["price"].str.contains(
    NON_NUMERIC_PRICE_PATTERN,
    case=False,
    na=False
)]


df["price"] = (
    df["price"]
    .str.replace("zł", "", regex=False)
    .str.replace("\xa0", "", regex=False)
    .str.replace(" ", "", regex=False)
)

df["price"] = pd.to_numeric(df["price"], errors='coerce')

df = df.dropna(subset=["price"])

df["price"].describe()

count    4.960000e+02
mean     6.035196e+05
std      2.364992e+05
min      5.000000e+03
25%      4.590000e+05
50%      5.517475e+05
75%      6.952358e+05
max      2.500000e+06
Name: price, dtype: float64

## 4. Area cleaning

Convert the apartment area from text format (e.g. "47 m²")
to a numeric value.


In [5]:
df = df[df["area"].notna()]
df["area"] = df["area"].astype(str)

df["area"] = (
    df["area"]
    .str.replace("m²", "", regex=False)
    .str.replace("\xa0", "", regex=False)
    .str.replace(" ", "", regex=False)
    .str.replace(",", ".", regex=False)
)

df["area"] = pd.to_numeric(df["area"], errors='coerce')
df = df.dropna(subset=["area"])
df = df[(df["area"] >= 20) & (df["area"] <= 200)]

df["area"].describe()


count    495.000000
mean      63.445495
std       23.832556
min       22.000000
25%       48.010000
50%       59.000000
75%       71.100000
max      153.000000
Name: area, dtype: float64

## 5. Price per square meter

Convert the price per square meter field to numeric format.

In [6]:
df = df[df["price_per_m2"].notna()]
df["price_per_m2"] = df["price_per_m2"].astype(str)

df["price_per_m2"] = (
    df["price_per_m2"]
    .str.replace("zł/m²", "", regex=False)
    .str.replace("\xa0", "", regex=False)
    .str.replace(" ", "", regex=False)
    .str.replace(",", ".", regex=False)
)

df["price_per_m2"] = pd.to_numeric(df["price_per_m2"], errors='coerce')
df = df.dropna(subset=["price_per_m2"])

df["price_per_m2"].describe()

count      495.000000
mean      9778.147475
std       2067.153957
min       5362.000000
25%       8436.500000
50%       9433.000000
75%      11200.000000
max      19844.000000
Name: price_per_m2, dtype: float64

## 6. Number of rooms

Extract the numeric number of rooms from the text field
(e.g. "3 pokoje").

In [7]:
df = df[df["rooms"].notna()]
df["rooms"] = df["rooms"].astype(str)

df["rooms"] = df["rooms"].str.extract(r"(\d+)")[0].astype(float)

df = df.dropna(subset=["rooms"])

df["rooms"].value_counts().sort_index()

rooms
1.0     20
2.0    181
3.0    210
4.0     62
5.0     21
6.0      1
Name: count, dtype: int64

## 7. Floor information

Floor is kept as a categorical field.
Missing values are expected for houses and some listings.


In [8]:
df["floor"].value_counts(dropna=False).head(10)

floor
1 piętro     113
parter       105
2 piętro      89
3 piętro      76
4 piętro      59
NaN           26
6 piętro       6
7 piętro       6
10 piętro      6
5 piętro       5
Name: count, dtype: int64

## 8. Missing values overview

Review the proportion of missing values in each column.


In [9]:
df.isna().mean().sort_values(ascending=False)


floor           0.052525
link            0.000000
price           0.000000
location        0.000000
area            0.000000
price_per_m2    0.000000
rooms           0.000000
offer_type      0.000000
dtype: float64

## 9. District extraction

Extract district name from the location field.
The logic handles listings both with and without street information.

In [14]:

def extract_district(location):
    if pd.isna(location):
        return None
        
    parts = [p.strip() for p in location.split(",")]

    if len(parts) > 1:
        first = parts[0].lower()
        if (first.startswith("ul.") or 
            first.startswith("pl.") or 
            first.startswith("rynek") or
            first.startswith("os.")):
            return parts[1]
    return parts[0] if parts else None

STREET_BLACKLIST = {
    "Śliczna",
    "Rajska",
    "kard. Stefana Wyszyńskiego",
    "Kołątaja",
    "pl. Józefa Piłsudskiego",
    "rynek Rynek"
}
df["district"] = df["location"].apply(extract_district)
df = df[~df["district"].isin(STREET_BLACKLIST)]
df["district"].value_counts()

district
Śródmieście                     95
Stare Miasto                    86
Armii Krajowej                  37
Zaodrze                         36
Kolonia Gosławicka              28
Szczepanowice - Wójtowa Wieś    27
Gosławice                       26
Malinka                         25
Nowa Wieś Królewska             22
Półwieś                         17
Chmielowice                     17
Nadodrze                        15
Chabry                          14
Osiny                           13
Groszowice                       9
Grotowice                        7
Malina                           4
Winów                            3
Bierkowice                       3
Czarnowąsy                       2
Grudzice                         1
Brzezie                          1
Wróblin                          1
Karczów                          1
Name: count, dtype: int64

## 10. Save cleaned dataset

Save the cleaned and deduplicated dataset for further analysis
and visualization in Power BI.

In [12]:
df.to_csv("../data/processed/opole_listings_clean.csv", index=False)